In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers import PreTrainedTokenizerFast, AutoModelForSequenceClassification, GPT2LMHeadModel


In [3]:
cls_peft_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none")

gen_peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none")

### 분류 모델

In [4]:
# 저장된 모델 및 토크나이저 로드
cls_path = '/models/kogpt2-classification-lora'
cls_model = AutoModelForSequenceClassification.from_pretrained(
      cls_path,
      num_labels=5,
      problem_type="multi_label_classification"
)
trained_cls_model = get_peft_model(cls_model, cls_peft_config)
trained_cls_tokenizer = PreTrainedTokenizerFast.from_pretrained(cls_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def predict_listener_empathy(input_text, model, tokenizer, num_classes=5, threshold=0.6):
    # 모델을 평가 모드로 전환
    model.eval()

    # 입력 문장 토큰화
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True, max_length=128)

    # 모델에 입력을 전달하여 로짓(logits)을 얻음
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 로짓에 시그모이드 적용하여 확률로 변환
    probabilities = torch.sigmoid(logits)
    # 임계값을 기준으로 이진화
    predictions = (probabilities > threshold).int()

    # 레이블 디코딩
    label_classes = [0, 1, 2, 3, 4]
    predicted_labels = [label_classes[i] for i in range(num_classes) if predictions[0][i] == 1]

    return predicted_labels


### 생성 모델

In [6]:
# 저장된 모델 및 토크나이저 로드
gen_path = '/models/kogpt2-chatbot'
gen_model = GPT2LMHeadModel.from_pretrained(gen_path)

trained_gen_model = get_peft_model(gen_model, gen_peft_config)
trained_gen_tokenizer = PreTrainedTokenizerFast.from_pretrained(gen_path)

In [21]:
def predict_answer(predicted_labels, input_text, model, tokenizer):
    # 모델을 평가 모드로 전환
    model.eval()
    # 입력 문장 토큰화
    condition = ' ,'.join(map(str, predicted_labels))
    inputs = f"{condition}를 중심으로 \"{input_text}\"에 대해 대답해줘"
    input_ids = tokenizer.encode(tokenizer.bos_token + inputs + tokenizer.eos_token, return_tensors='pt')

    # 모델 추론
    outputs = model.generate(input_ids, max_length=50, repetition_penalty=2.0, num_beams=5, early_stopping=True)
    output_text = trained_gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return output_text

## 추론

In [22]:
# 예제 입력 문장
input_text = "오늘 기분은 어때?"

# 분류 결과 추론
# threshold 잘 설정해야
predicted_labels = predict_listener_empathy(input_text, trained_cls_model, trained_cls_tokenizer, threshold=0.5)
print(f"Predicted labels: {predicted_labels}")

Predicted labels: [0, 4]


In [23]:
print(predict_answer(predicted_labels, input_text, trained_gen_model, trained_gen_tokenizer))

0,4를 중심으로 "오늘 기분은 어때?"에 대해 대답해줘
